#### Let's import all the required libraries!!

In [115]:
import numpy as np # library for Array/vector data

import pandas as pd # library for data analysis
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)
import requests # library to handle requests
#from pandas.io.json import json_normalize # tranform JSON file into a pandas dataframe
from pandas import json_normalize
import folium # map rendering library

print('Libraries imported.')

Libraries imported.


#### Using Foursquare for pulling location Data

In [116]:
CLIENT_ID = 'ODW53TD40J43SX3IO5J1TO2DTHKWKJRGKWRXPEHXTAHJR33M' # Foursquare ID
CLIENT_SECRET = '****' # Foursquare Secret
VERSION = '20180605' # Foursquare API version

print('Foursquare credentials:')
print('CLIENT_ID: ' + CLIENT_ID)

Foursquare credentials:
CLIENT_ID: ODW53TD40J43SX3IO5J1TO2DTHKWKJRGKWRXPEHXTAHJR33M


#### Pull Indian Restaurants Info for 4 Major US Cities

In [117]:
LIMIT = 500 # Maximum is 100
cities = [ 'Chicago, IL', 'Miami, FL', 'New York, NY', 'Los Angeles, CA']
results = {}
for city in cities:
    url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&near={}&limit={}&categoryId={}'.format(
        CLIENT_ID, 
        CLIENT_SECRET, 
        VERSION, 
        city,
        LIMIT,
        "4bf58dd8d48988d10f941735") # INDIAN RESTAURANTS CATEGORY ID
    results[city] = requests.get(url).json()

In [118]:
df_venues={}
for city in cities:
    venues = json_normalize(results[city]['response']['groups'][0]['items'])
    df_venues[city] = venues[['venue.name', 'venue.location.address', 'venue.location.lat', 'venue.location.lng']]
    df_venues[city].columns = ['Name', 'Address', 'Lat', 'Lng']

#### Foursquare API will only show the nearest 100 venues in the city. Lets look at their density through Maps.

In [119]:
maps = {}
for city in cities:
    city_lat = np.mean([results[city]['response']['geocode']['geometry']['bounds']['ne']['lat'],
                        results[city]['response']['geocode']['geometry']['bounds']['sw']['lat']])
    city_lng = np.mean([results[city]['response']['geocode']['geometry']['bounds']['ne']['lng'],
                        results[city]['response']['geocode']['geometry']['bounds']['sw']['lng']])
    maps[city] = folium.Map(location=[city_lat, city_lng], zoom_start=11)

    # add markers to map
    for lat, lng, label in zip(df_venues[city]['Lat'], df_venues[city]['Lng'], df_venues[city]['Name']):
        label = folium.Popup(label, parse_html=True)
        folium.CircleMarker(
            [lat, lng],
            radius=5,
            popup=label,
            color='darkblue',
            fill=True,
            fill_color='#3186cc',
            fill_opacity=0.6,
            parse_html=False).add_to(maps[city])  
    print(f"Total number of Indian Restaurants in {city} = ", results[city]['response']['totalResults'])
    print("Showing Top 100")

Total number of Indian Restaurants in Chicago, IL =  169
Showing Top 100
Total number of Indian Restaurants in Miami, FL =  64
Showing Top 100
Total number of Indian Restaurants in New York, NY =  247
Showing Top 100
Total number of Indian Restaurants in Los Angeles, CA =  233
Showing Top 100


### Chicago Neighborhood Map

In [120]:
maps[cities[0]]

### Miami Neighborhood Map

In [121]:
maps[cities[1]]

### New York City Neighborhood Map

In [122]:
maps[cities[2]]

### Los Angeles Neighborhood Map

In [123]:
maps[cities[3]]

##### New York and Los Angeles are more dense cities with Indian Restaurants compared to Chicago. 
But, Let's have a concrete measure of this density.

To achieve this, I will get the mean location of the Indian restaurants which are near to most of them.
Next I will take the average of the distance of the venues to the mean coordinates.

In [126]:
maps = {}
for city in cities:
    city_lat = np.mean([results[city]['response']['geocode']['geometry']['bounds']['ne']['lat'],
                        results[city]['response']['geocode']['geometry']['bounds']['sw']['lat']])
    city_lng = np.mean([results[city]['response']['geocode']['geometry']['bounds']['ne']['lng'],
                        results[city]['response']['geocode']['geometry']['bounds']['sw']['lng']])
    maps[city] = folium.Map(location=[city_lat, city_lng], zoom_start=10.5)
    venues_mean_coord = [df_venues[city]['Lat'].mean(), df_venues[city]['Lng'].mean()] 
    # add markers to map
    for lat, lng, label in zip(df_venues[city]['Lat'], df_venues[city]['Lng'], df_venues[city]['Name']):
        label = folium.Popup(label, parse_html=True)
        folium.CircleMarker(
            [lat, lng],
            radius=5,
            popup=label,
            color='darkblue',
            fill=True,
            fill_color='#3186cc',
            fill_opacity=0.6,
            parse_html=False).add_to(maps[city])
        folium.PolyLine([venues_mean_coord, [lat, lng]], color="darkgreen", weight=1.5, opacity=0.5).add_to(maps[city])
    
    label = folium.Popup("Mean Co-ordinate", parse_html=True)
    folium.CircleMarker(
        venues_mean_coor,
        radius=10,
        popup=label,
        color='red',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.6,
        parse_html=False).add_to(maps[city])

    print(city)
    print("Mean Distance from Mean coordinates")
    print(np.mean(np.apply_along_axis(lambda x: np.linalg.norm(x - venues_mean_coord),1,df_venues[city][['Lat','Lng']].values)))

Chicago, IL
Mean Distance from Mean coordinates
0.05479529209050545
Miami, FL
Mean Distance from Mean coordinates
0.1009260481763159
New York, NY
Mean Distance from Mean coordinates
0.025017747308247504
Los Angeles, CA
Mean Distance from Mean coordinates
0.09705193629571783


### Chicago Neighborhood Map

In [127]:
maps[cities[0]]

### Miami Neighborhood Map

In [128]:
maps[cities[1]]

### New York Neighborhood Map

In [129]:
maps[cities[2]]

### Los Angeles Neighborhood Map

In [130]:
maps[cities[3]]

### New York is the Clear Winner. Our Tourist's best interest would be to book a hotel near the mean coordinate to surround himself with so many Indian Restaurants!!